# 🏥 MedGuard Triage Copilot – Development Notebook

**Three-stage, safety-first triage pipeline:**
1. **MedASR** (Whisper) → Speech-to-Text
2. **Gemma 7B-IT** → Clinical Structurer (entity extraction, negation, uncertainty)
3. **MedGemma 4B-IT** → Triage Reasoner (risk scoring, red flags, safety notes)


### 1. Setup & Imports


In [ ]:
!pip install -q transformers torch pydantic accelerate sentencepiece requests PyYAML python-dotenv huggingface_hub


  Using cached torch-2.10.0-cp312-cp312-manylinux_2_28_x86_64.whl.metadata (31 kB)
  Using cached accelerate-1.12.0-py3-none-any.whl.metadata (19 kB)
  Using cached sentencepiece-0.2.1-cp312-cp312-manylinux_2_27_x86_64.manylinux_2_28_x86_64.whl.metadata (10 kB)
  Using cached setuptools-80.10.2-py3-none-any.whl.metadata (6.6 kB)
  Using cached sympy-1.14.0-py3-none-any.whl.metadata (12 kB)
  Using cached networkx-3.6.1-py3-none-any.whl.metadata (6.8 kB)
  Using cached cuda_bindings-12.9.4-cp312-cp312-manylinux_2_24_x86_64.manylinux_2_28_x86_64.whl.metadata (2.6 kB)
  Using cached nvidia_cuda_nvrtc_cu12-12.8.93-py3-none-manylinux2010_x86_64.manylinux_2_12_x86_64.whl.metadata (1.7 kB)
  Using cached nvidia_cuda_runtime_cu12-12.8.90-py3-none-manylinux2014_x86_64.manylinux_2_17_x86_64.whl.metadata (1.7 kB)
  Using cached nvidia_cuda_cupti_cu12-12.8.90-py3-none-manylinux2014_x86_64.manylinux_2_17_x86_64.whl.metadata (1.7 kB)
  Using cached nvidia_cudnn_cu12-9.10.2.21-py3-none-manylinux_2_27

In [ ]:
import sys, os
sys.path.insert(0, os.path.abspath(".."))

import json
from dotenv import load_dotenv
load_dotenv("../.env")

# Verify token is set
assert os.environ.get("HF_API_TOKEN") or os.environ.get("HF_TOKEN"), "Set HF_API_TOKEN in .env"
print("✅ Setup complete")


In [ ]:
# Import pipeline components
from models.extraction.schema_definition import (
    StructuredClinicalData, TriageOutput, PipelineResult,
    Symptom, NegationStatus
)
from models.extraction.gemma_structurer import GemmaStructurer
from models.extraction.uncertainty_engine import UncertaintyEngine
from models.triage.medgemma_reasoner import MedGemmaReasoner
from models.triage.red_flag_engine import RedFlagEngine
from models.triage.risk_scorer import RiskScorer
from pipelines.text_pipeline import TextPipeline
from pipelines.structured_pipeline import StructuredPipeline
from core.router import MedGuardRouter

print("✅ All modules imported")


In [ ]:
# Optional: Override endpoint URLs for dedicated inference endpoints
# os.environ["STRUCTURER_ENDPOINT_URL"] = "https://your-gemma-endpoint.huggingface.cloud"
# os.environ["TRIAGE_ENDPOINT_URL"] = "https://your-medgemma-endpoint.huggingface.cloud"


### 2. Schema Definitions
The pipeline uses Pydantic schemas to enforce structure at every stage.


In [ ]:
# Inspect the output schemas
print("=== StructuredClinicalData (Stage 2 output) ===")
print(json.dumps(StructuredClinicalData.model_json_schema(), indent=2)[:800])

print("\n=== TriageOutput (Stage 3 output) ===")
print(json.dumps(TriageOutput.model_json_schema(), indent=2)[:600])


### 3. Rule-Based Red Flag Engine (runs independently of models)


# Test the red flag engine with sample structured data
red_flag_engine = RedFlagEngine()

sample_structured = {
    "chief_complaint": "Crushing chest pain",
    "symptoms": [
        {"name": "crushing chest pain", "negation": "affirmed", "severity": "severe"},
        {"name": "diaphoresis", "negation": "affirmed"},
        {"name": "nausea", "negation": "negated"},
    ],
    "risk_factors": ["hypertension", "smoker"],
}

flags = red_flag_engine.scan(sample_structured)
min_urgency = red_flag_engine.get_min_urgency(flags)

print(f"🚩 Red flags detected: {len(flags)}")
for f in flags:
    print(f"   [{f['category']}] {f['flag']} → {f['recommended_action']}")
print(f"\n⚠️  Minimum urgency from rules: {min_urgency}")


In [ ]:
### 4. Uncertainty Engine


# Test uncertainty analysis on sparse data
uncertainty_engine = UncertaintyEngine()

sparse_data = {
    "chief_complaint": "headache",
    "symptoms": [
        {"name": "headache", "negation": "uncertain", "confidence": 0.3},
    ],
    "demographics": {},
    "extraction_confidence": 0.6,
    "missing_information": [],
    "uncertainty_notes": [],
}

analyzed = uncertainty_engine.analyze(sparse_data)
print(f"📊 Extraction confidence: {analyzed['extraction_confidence']}")
print(f"📋 Missing information ({len(analyzed['missing_information'])}):")
for m in analyzed["missing_information"][:5]:
    print(f"   • {m}")
print(f"\n⚠️  Uncertainty notes ({len(analyzed['uncertainty_notes'])}):")
for n in analyzed["uncertainty_notes"][:5]:
    print(f"   [{n['impact']}] {n['field']}: {n['reason']}")


In [ ]:
### 5. Stage 2 – Gemma Clinical Structurer
Extracts structured entities from free-text using Gemma 7B Instruct.


# Initialize the Gemma structurer
structurer = GemmaStructurer()

# Test with free-text patient intake
patient_text = """
58-year-old male presenting with crushing chest pain radiating to left arm 
for 45 minutes. History of hypertension, smoker. Diaphoretic, BP 160/95, 
HR 110, SpO2 94%. Denies nausea. No known drug allergies.
"""

structured_data = structurer.structure(patient_text)
print("=== Stage 2: Structured Clinical Data ===")
print(json.dumps(structured_data, indent=2, default=str)[:1500])


In [ ]:
### 6. Stage 3 – MedGemma Triage Reasoner
Performs safety-focused triage reasoning on structured data.


# Initialize triage reasoner
reasoner = MedGemmaReasoner()

# Run triage on structured data (using sample if structurer hasn't run)
with open("../data/samples/sample_structured.json") as f:
    sample_structured = json.load(f)

triage_raw = reasoner.reason(sample_structured)
print("=== Stage 3: Raw Triage Output ===")
print(json.dumps(triage_raw, indent=2, default=str)[:1200])


In [ ]:
### 7. Risk Scorer – Merging Model + Rule-Based Signals


# Combine model triage + rule-based red flags
risk_scorer = RiskScorer()
rule_flags = red_flag_engine.scan(sample_structured)
rule_min = red_flag_engine.get_min_urgency(rule_flags)

final_triage, overrides = risk_scorer.score(
    triage_output=triage_raw,
    rule_flags=rule_flags,
    rule_min_urgency=rule_min,
    extraction_confidence=sample_structured.get("extraction_confidence", 0.5),
)

print("=== Final Scored Triage ===")
print(f"Urgency: Level {final_triage['urgency_level']} – {final_triage['urgency_label']}")
print(f"Confidence: {final_triage['confidence_score']} ({final_triage['confidence_band']})")
print(f"\nRed flags: {len(final_triage.get('red_flags', []))}")
print(f"Safety overrides: {overrides}")
print(f"\nSafety note: {final_triage.get('safety_note', '')}")


### 8. Full Pipeline – End-to-End


In [ ]:
# Use the MedGuardRouter for one-call triage
router = MedGuardRouter()

# Test case: High-risk cardiac
result = router.triage(
    "58-year-old male with crushing chest pain radiating to left arm for 45 minutes. "
    "History of hypertension, smoker. Diaphoretic, BP 160/95, HR 110, SpO2 94%."
)

print("=== Full Pipeline Result ===")
print(f"ID: {result['id']}")
print(f"Input type: {result['input_type']}")
triage = result.get("triage_output", {})
print(f"Urgency: Level {triage.get('urgency_level')} – {triage.get('urgency_label')}")
print(f"Confidence: {triage.get('confidence_score')} ({triage.get('confidence_band')})")
print(f"Red flags: {len(triage.get('red_flags', []))}")
print(f"Overrides: {result.get('safety_overrides_applied', [])}")
print(f"Errors: {result.get('pipeline_errors', [])}")


### 9. Multi-Case Demo


In [ ]:
# Test multiple scenarios
demo_cases = [
    {
        "label": "🔴 Cardiac Emergency",
        "input": "58yo male, crushing chest pain radiating to left arm, 45 min, smoker, diaphoretic"
    },
    {
        "label": "🟢 Minor URI",
        "input": "28yo female, runny nose and mild cough for 3 days, no fever, feeling okay"
    },
    {
        "label": "🟡 Abdominal Pain",
        "input": "45yo female, RLQ pain for 12 hours with nausea and low-grade fever. No prior surgeries."
    },
    {
        "label": "🔴 Suicidal Ideation",
        "input": "35yo male, acute distress, reports wanting to die, has a plan involving medications"
    },
]

COLORS = {5: "🔴", 4: "🟠", 3: "🟡", 2: "🔵", 1: "🟢"}

for case in demo_cases:
    print(f"\n{'='*50}")
    print(f"  {case['label']}")
    print(f"{'='*50}")
    try:
        result = router.triage(case["input"])
        t = result.get("triage_output", {})
        level = t.get("urgency_level", "?")
        icon = COLORS.get(level, "⚪")
        print(f"  {icon} Level {level} – {t.get('urgency_label', '?')}")
        print(f"  Confidence: {t.get('confidence_score', '?')}")
        flags = t.get("red_flags", [])
        if flags:
            print(f"  🚩 Red flags: {[f.get('flag', f) if isinstance(f, dict) else f for f in flags]}")
        overrides = result.get("safety_overrides_applied", [])
        if overrides:
            print(f"  🔒 Overrides: {overrides}")
    except Exception as e:
        print(f"  ❌ Error: {e}")


RAW OUTPUT: 

You are a clinical triage copilot designed to support healthcare professionals.
You assess urgency, identify red flags, and suggest next steps.
You do NOT diagnose.
You prioritize safety and transparency.



Analyze the following patient intake for triage purposes.

Patient intake:
{"age": 58, "sex": "male", "chief_complaint": "Chest pain", "symptoms": ["pressure-like chest pain", "shortness of breath"], "duration": "45 minutes", "risk_factors": ["hypertension", "smoker"]}

Return ONLY valid J


ValueError: Unclosed JSON object